# Read in CSV from AWS

In [ ]:
%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
Toys_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)
Toys_df.show()

In [ ]:
%pyspark
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games_df.show()

# Count rows

In [ ]:
%pyspark

Toys_df.count()

In [ ]:
%pyspark

video_games_df.count()

# Merge DataFrames

In [ ]:
%pyspark
df_concat = video_games_df.union(Toys_df)
df_concat.count()

# Clean up DataFrames to match tables

In [ ]:
%pyspark
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = df_concat.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates()
review_id_df.show()

# Products DataFrame

In [ ]:
%pyspark

products_df = df_concat.select(["product_id", "product_title", "product_category"]).drop_duplicates()
products_df.show()

# Customers DataFrame

In [ ]:
%pyspark

customers_df = df_concat.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

# Reviews DataFrame

In [ ]:
%pyspark

reviews_df = df_concat.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show()

# Vine DataFrame

In [ ]:
%pyspark

vine_df = df_concat.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

# Push to AWS RDS instance.

In [ ]:
%pyspark
# Configuration for RDS instance
mode="append"
jdbc_url="jdbc:postgresql://mypostgresdb.cr2hoksxzp5a.us-east-2.rds.amazonaws.com:5432/big_data_db"
config = {"user":"<>", 
          "password": "<>", 
          "driver":"org.postgresql.Driver"}

# Write products_df to table in RDS

In [ ]:
%pyspark

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

# Write customers_df to table in RDS

In [ ]:
%pyspark

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

# Write review_id_df to table in RDS

In [ ]:
%pyspark

reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

# Write vine_df to table in RDS

In [ ]:
%pyspark

vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)